In [3]:
#dependencies 
from api_key import Auth_Token
import requests
import json
from pandas.io.json import json_normalize
from flatten_json import flatten
import pandas as pd
import regex 

In [4]:
#Options for Params
    #"q"
search = 'use the input function ->' #input() if creating a dasboard. 
    # "year"
model_year = 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023
    # "category"
cat = "mountain"
    # "subcategory"
sub_cat = "trail", "crosscountry", "enduro",
    # "makerId"
Brand = "salsa", "surly", "trek", "specialized", "giant", "cannondale", "kona", "rockymountain", "santacruz", "norco", "yeti", "niner", "cannondale", "scott", "orbea", "pivot", "cube", "bmc", "yt", "fezzari","canyon","ibis","marin","diamondback"
    # "isEbike"
eBike = "false"
    #"isFrameset"
frameset = "false"
    #"include"
include = "*"

MTB = []

params = {
    "limit" : 200,
    "cursor" : 'start',
    "makerId" : Brand,
    "category" : cat,
    "subcategory" : sub_cat,  
    "include" : include,
    "isFrameset" : frameset,

}

while True: 
    BASE_URL = 'https://api.99spokes.com/v1/bikes'
    headers = {f'Authorization': "Bearer {}".format(Auth_Token)}
    response = requests.get(BASE_URL, headers=headers, params=params)
    bikes = response.json()
    if bikes['nextCursor'] == None:
        print("Complete")
        break
    MTB.extend(bikes['items'])
    nextCursor = bikes['nextCursor']
    params = {
    "limit" : 200,
    "cursor" : nextCursor,
    "makerId" : Brand,
    "category" : cat,
    "subcategory" : sub_cat,  
    "include" : include,
    "isFrameset" : frameset,
}


Complete


In [5]:
#flatten all the data and place every list into columns. 
data = MTB
dict_flattened = (flatten(record, '.') for record in data)
df = pd.DataFrame(dict_flattened)
print(f"Data Returned this many bikes:", len(df))

Data Returned this many bikes: 11600


In [6]:
df['sizes.2.geometry.computed.rakeMM']

0        43.0
1        41.0
2        42.0
3        44.0
4        42.0
         ... 
11595     NaN
11596     NaN
11597     NaN
11598     NaN
11599     NaN
Name: sizes.2.geometry.computed.rakeMM, Length: 11600, dtype: float64

In [7]:
#clean up the data to better analyze (Example:  Removing the inch callout in tire size)

df["components.tires.width"] = df["components.tires.width"].str.replace('"',"", regex=True).str.replace('c',"", regex=True)

# df.to_csv("check.csv")

In [8]:
#reduce down all the unwanted columns and rename them. 
reduced_df = df[['maker', 'year', 'model', 'family', 'category', 'subcategory', 'isEbike', 'isFrameset',
                'prices.0.currency', 'prices.0.amount', 'priceHistory.0.history.0.date', 
                'weight.weightKG', 'components.frame.material', 'colors.0.name',
                'suspension.configuration', "suspension.front.travelMM",
                'components.fork.maker', 'components.fork.model',  
                'wheels.configuration', 'wheels.kinds.0', 
                'shifting.kind', 'gearing.rear.count', 'components.rearDerailleur.model', 'components.rearDerailleur.maker',  
                'gearing.front.count', 'components.frontDerailleur.display', 'components.frontDerailleur.maker',
                'components.bottomBracket.description', 'components.bottomBracket.standard', 'components.bottomBracket.shellWidthMM', 'components.crank.description', 
                'components.chain.display', 'components.chain.maker', 'components.cassette.description', 'components.cassette.maker', 
                'components.shifters.display', 'components.shifters.maker', 
                'components.brakes.description', 'components.brakes.maker', 'components.brakes.kind', 'components.diskRotors.description', 'components.diskRotors.maker', 
                'components.rims.description', 'components.rims.maker', 'components.rims.material', 'components.rims.innerWidthMM',
                'components.frontHub.description', 'components.frontHub.maker', 'components.rearHub.description', 'components.rearHub.maker', 
                'components.tires.description', 'components.tires.maker', 'components.tires.width',
                'components.handlebar.description', 'components.handlebar.kind', 'components.saddle.description', 'components.saddle.maker', 
                'components.seatpost.description', 'components.seatpost.kind', 'components.seatpost.material',
                'components.headset.description', 'components.pedals.description',
                'sizes.2.geometry.source.seatTubeAngle', 'sizes.2.geometry.source.headTubeAngle', 'sizes.2.geometry.source.chainstayLengthMM', 
                'sizes.2.geometry.source.bottomBracketDropMM', 'sizes.2.geometry.computed.stackReachRatio', 
                'sizes.2.geometry.computed.rakeMM', 'sizes.2.geometry.computed.trailMM',
                'url'
                ]]

mtb_df = pd.DataFrame(reduced_df.rename(columns={'maker':'Brand', 'year':'Model_Year', 'model':'Model', 'family':'Platform', 'category':'Category', 'subcategory':'Sub_Cat', "isEbike": "Ebike", 'isFrameset':"Frameset",
                'prices.0.currency':'Currency', 'prices.0.amount':'Price', 'priceHistory.0.history.0.date':'Price_Date', 
                'weight.weightKG':'Weight(KG)', 'components.frame.material':'Frame_Material', 'colors.0.name':'Color',
                'suspension.configuration':'Suspension_Type', "suspension.front.travelMM":"Front_Travel",
                'components.fork.maker':'Fork_Brand', 'components.fork.model':'Fork_Brand',  
                'wheels.configuration':'Wheel_Size_Config', 'wheels.kinds.0':'Wheel_Size', 
                'shifting.kind':'Shift_Type', 'gearing.rear.count':'Drivetrain_Speeds', 'components.rearDerailleur.model':"R_Dereailluer_Model", 'components.rearDerailleur.maker':'R_Derailluer_Brand',  
                'gearing.front.count':'Front_Gearing', 'components.frontDerailleur.display':'F_Derailluer_Model', 'components.frontDerailleur.maker':'F_Derailluer_Brand',
                'components.bottomBracket.description':'BB_Model', 'components.bottomBracket.standard':'BB_Standard', 'components.bottomBracket.shellWidthMM':'BB_Width', 'components.crank.description':"Crank_Model", 
                'components.chain.display':'Chain_Model', 'components.chain.maker':'Chain_Brand', 'components.cassette.description':'Cassette_Model', 'components.cassette.maker':'Cassette_Brand', 
                'components.shifters.display':'Shifter_Model', 'components.shifters.maker':'Shifter_Brand', 
                'components.brakes.description':'Brake_Model', 'components.brakes.maker':'Brake_Brand', 'components.brakes.kind':'Brake_Type', 
                'components.diskRotors.description':'Rotor_Model', 'components.diskRotors.maker':'Rotor_Brand', 
                'components.rims.description':'Rim_Model', 'components.rims.maker':'Rim_Brand', 'components.rims.material':'Rim_Material', 'components.rims.innerWidthMM':'Rim_Width',
                'components.frontHub.description':'F_Hub_Model', 'components.frontHub.maker':'F_Hub_Brand', 'components.rearHub.description':'R_Hub_Model', 'components.rearHub.maker':'R_Hub_Brand', 
                'components.tires.description':'Tire_Model', 'components.tires.maker':'Tire_Brand', 'components.tires.width':"Tire_Width",
                'components.handlebar.description':'Handlebar_Model', 'components.handlebar.kind':'Handlebar_Type', 'components.saddle.description':'Saddle_Model', 'components.saddle.maker':'Saddle_Brand', 
                'components.seatpost.description':'Seatpost_Model', 'components.seatpost.kind':'Seatpost_Type', 'components.seatpost.material':'Seatpost_Material',
                'components.headset.description':'Headset', 'components.pedals.description':'Pedals',
                'sizes.2.geometry.source.seatTubeAngle':'ST_Angle', 'sizes.2.geometry.source.headTubeAngle':'HT_Angle', 'sizes.2.geometry.source.chainstayLengthMM':'CS_length', 
                'sizes.2.geometry.source.bottomBracketDropMM':'BB_Drop', 'sizes.2.geometry.computed.stackReachRatio':'StackReach_Ratio', 
                'sizes.2.geometry.computed.rakeMM':'Rake', 'sizes.2.geometry.computed.trailMM':'Trail',
                'url':'URL'
        
                }))     
mtb_df                

,Brand,Model_Year,Model,Platform,Category,Sub_Cat,Ebike,Frameset,Currency,Price,...,Headset,Pedals,ST_Angle,HT_Angle,CS_length,BB_Drop,StackReach_Ratio,Rake,Trail,URL
0,Canyon,2023,Spectral 27 FW,Spectral,mountain,enduro,False,False,EUR,3499.0,...,NaN,NaN,77.0,64.5,432.0,15.0,1.346,43.0,122.0,https://99spokes.com/bikes/canyon/2023/spectra...
1,Canyon,2023,Spectral 29 FW,Spectral,mountain,enduro,False,False,EUR,3499.0,...,NaN,NaN,76.5,64.0,437.0,30.0,1.265,41.0,137.0,https://99spokes.com/bikes/canyon/2023/spectra...
2,Canyon,2023,Spectral 27.5 CF 7,Spectral,mountain,enduro,False,False,USD,3999.0,...,NaN,NaN,76.5,64.5,432.0,20.0,1.354,42.0,123.0,https://99spokes.com/bikes/canyon/2023/spectra...
3,Canyon,2023,Spectral 5,Spectral,mountain,enduro,False,False,USD,3299.0,...,NaN,NaN,74.0,66.0,430.0,19.0,1.400,44.0,110.0,https://99spokes.com/bikes/canyon/2023/spectral-5
4,Canyon,2023,Spectral 27.5 CF 9,Spectral,mountain,enduro,False,False,USD,5999.0,...,NaN,NaN,76.5,64.5,432.0,20.0,1.354,42.0,123.0,https://99spokes.com/bikes/canyon/2023/spectra...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11595,Norco,2011,Jubei SL,Jubei,mountain,crosscountry,False,False,USD,2199.0,...,NaN,NaN,73.0,72.0,440.0,NaN,NaN,NaN,NaN,https://99spokes.com/bikes/norco/2011/jubei-sl
11596,Norco,2011,Jubei 1,Jubei,mountain,crosscountry,False,False,USD,1782.0,...,FSA ZS-4D w/12mm conical spacer,Alloy silver body w/black alloy cage,73.0,71.0,445.0,NaN,NaN,NaN,NaN,https://99spokes.com/bikes/norco/2011/jubei-1
11597,Norco,2011,Jubei 2,Jubei,mountain,crosscountry,False,False,USD,1214.0,...,FSA ZS-4D w/12mm conical spacer,Alloy silver body w/black alloy cage,73.0,71.0,445.0,NaN,NaN,NaN,NaN,https://99spokes.com/bikes/norco/2011/jubei-2
11598,Norco,2011,Fireball,Fireball,mountain,crosscountry,False,False,USD,1500.0,...,FSA ZS-4D w/12mm conical spacer,Alloy silver body w/black alloy cage,73.0,70.5,425.0,NaN,NaN,NaN,NaN,https://99spokes.com/bikes/norco/2011/fireball


In [10]:
mtb_df  = mtb_df.dropna(subset=['Frame_Material'])

mtb_df.to_csv("MTB.csv")  